In [171]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from sentence_transformers import SentenceTransformer

/home/mark/.local/share/virtualenvs/gloo-hackathon-2023-vR4QBjT8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [100]:
model = spacy.load('en_core_web_lg')

In [102]:
files = Path('data')

In [147]:
data = {}
#iterate over files
for file in files.iterdir():
    if file.stem == 'vref':
        with open(file , 'r') as f:
            current_text = f.readlines()
            current_text = [x.strip() for x in current_text]
            vrefs = current_text
            continue
    with open(file , 'r') as f:
        current_text = f.readlines()
        current_text = [x.strip() for x in current_text]
        data[file.stem] = current_text

In [162]:
excluded_words = ['the', 'a', ',', '.', 'in', 'to', 'of', '-', '“', '”']

In [165]:
current_text = 'en-NASB'
for line_no in range(0, 20):
    words = {}
    matches = {}

    for current_text in data:
        words[current_text] = []
        d = model(data[current_text][line_no])
        for token in d:
            # if token.pos_ == 'VERB':
            words[current_text].append({'token': str(token), 'vector': token.vector})
                
                # print(type(token.vector))
    for word in words[current_text]:
        if word['token'] in excluded_words:
            continue
        # print(f'{word["token"]=}')
        # print(f'{word["vector"]=}')

        # print(f'Current text: {current_text}, word: {word["token"]}')
        if word['token'] in matches:
            continue
            
        top_similarities = np.array([])

        for other_text in data:
            # print(f'{words[line_no][other_text][0]["vector"]=}')
            similarities = cosine_similarity([word['vector']], [other_word['vector'] for other_word in words[other_text]])
            # print(f'Other text: {other_text}, similarity: {np.max(similarities)}, word: {words[line_no][other_text][np.argmax(similarities)]["token"]}')
            top_similarities = np.append(top_similarities, np.max(similarities))
            # print(f'{top_similarities=}')
        if np.mean(top_similarities) > 0.9:
            # matches[word['token']] = top_similarities
            print(f'{vrefs[line_no]}: {word["token"]}: {np.mean(top_similarities)}')
            matches[word['token']] = np.mean(top_similarities)

GEN 1:1: In: 1.0
GEN 1:1: beginning: 1.0000001192092896
GEN 1:1: God: 0.9999998807907104
GEN 1:1: created: 0.9999999403953552
GEN 1:1: heavens: 0.9408477246761322
GEN 1:1: and: 0.9390776654084524
GEN 1:1: earth: 0.9316087166468302
GEN 1:2: earth: 1.0
GEN 1:2: was: 1.0000001192092896
GEN 1:2: formless: 0.9440080722173055
GEN 1:2: and: 0.9999999403953552
GEN 1:2: darkness: 0.9999998807907104
GEN 1:2: deep: 0.9109650651613871
GEN 1:2: God: 0.9999998807907104
GEN 1:2: over: 1.0
GEN 1:2: waters: 0.9606208701928457
GEN 1:3: Then: 0.9058253765106201
GEN 1:3: God: 0.9999998807907104
GEN 1:3: Let: 1.0000001192092896
GEN 1:3: there: 1.0000001192092896
GEN 1:3: be: 0.9999998807907104
GEN 1:3: light: 1.0
GEN 1:3: and: 0.9999999403953552
GEN 1:4: God: 0.9999998807907104
GEN 1:4: saw: 1.0000001192092896
GEN 1:4: that: 0.9421694974104563
GEN 1:4: light: 1.0
GEN 1:4: was: 1.0000001192092896
GEN 1:4: good: 0.9052263995011648
GEN 1:4: separated: 0.9999997615814209
GEN 1:4: from: 1.0
GEN 1:4: darkness: 0

Similarity: 0.6310964822769165
